## SECTION 1

In [1]:
# ============================================================================
# SECTION 1: ENVIRONMENT SETUP (ROBUST, PY3.12-FRIENDLY)
# ============================================================================

import sys, subprocess, importlib, os

def pipi(*pkgs):
    # Force reinstall + no cache to avoid stale wheels
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "--force-reinstall", "--no-cache-dir", *pkgs])

print("Installing pinned, compatible versions …")
# Torch: keep your existing CUDA build. If you don't have torch yet, uncomment the torch trio below.
# pipi("torch==2.2.2", "torchaudio==2.2.2", "torchvision==0.17.2")

# Pin NumPy 2.x and libs that are built against it
pipi(
    "numpy==2.1.1",
    "pandas==2.2.3",
    "scikit-learn==1.5.2",
    "matplotlib==3.9.2",
    "transformers==4.44.2",
    "accelerate==0.34.2",
    "datasets==2.21.0",
)

# --- Import order matters; import numpy FIRST to catch ABI issues clearly
import numpy as np
print("NumPy:", np.__version__)

# Now the rest
import torch, transformers, datasets, sklearn, pandas as pd, matplotlib, importlib

print("\n=== VERSION CHECK ===")
print("torch          :", getattr(torch, "__version__", "n/a"))
print("transformers   :", transformers.__version__)
print("accelerate     :", importlib.import_module("accelerate").__version__)
print("datasets       :", datasets.__version__)
print("scikit-learn   :", sklearn.__version__)
print("pandas         :", pd.__version__)
print("numpy          :", np.__version__)
print("matplotlib     :", matplotlib.__version__)

# Sanity for TrainingArguments modern kwargs
from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.26.0"), \
    "Transformers too old for `evaluation_strategy`."

# If NumPy was previously imported in this session, you may still have stale .so’s in memory.
# Simple guard: if you see an ABI error above, Restart runtime and run this cell again first.
print("\nCUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Count:", torch.cuda.device_count())
    print("Current CUDA Device:", torch.cuda.get_device_name(0))


Installing pinned, compatible versions …
NumPy: 1.26.4

=== VERSION CHECK ===
torch          : 2.9.0+cu128
transformers   : 4.44.2
accelerate     : 0.34.2
datasets       : 2.21.0
scikit-learn   : 1.5.2
pandas         : 2.2.3
numpy          : 1.26.4
matplotlib     : 3.9.2

CUDA Available: True
CUDA Device Count: 1
Current CUDA Device: Tesla T4


### SECTION 1.5


In [2]:
# ============================================================================
# SECTION 1.5: VERSION CHECK + TRAININGARGUMENTS COMPATIBILITY SHIM
# ============================================================================

import inspect, importlib, sys
import transformers as _tf

print("Transformers version loaded in memory:", _tf.__version__)

def _supported_kwargs_of_training_args():
    # Build the set of supported __init__ kwargs for the loaded TrainingArguments
    try:
        from transformers import TrainingArguments
        sig = inspect.signature(TrainingArguments.__init__)
        return set(sig.parameters.keys())
    except Exception as e:
        print("[Compat] Could not inspect TrainingArguments:", e)
        return set()

_SUPPORTED_TA_KEYS = _supported_kwargs_of_training_args()
print("Sample of supported TrainingArguments kwargs:", sorted(list(_SUPPORTED_TA_KEYS))[:12], "...")

def make_training_args_compat(**kwargs):
    """
    Create TrainingArguments while dropping any kwargs unsupported by the loaded transformers version.
    Prints what was ignored so you know if your runtime is old.
    """
    from transformers import TrainingArguments
    filtered = {k: v for k, v in kwargs.items() if k in _SUPPORTED_TA_KEYS}
    ignored = [k for k in kwargs.keys() if k not in _SUPPORTED_TA_KEYS]
    if ignored:
        print("[Compat] Ignored unsupported TrainingArguments keys:", ignored)
    return TrainingArguments(**filtered)

def get_early_stopping_callbacks(patience: int):
    """Return EarlyStoppingCallback if available; otherwise return []."""
    try:
        from transformers import EarlyStoppingCallback
        return [EarlyStoppingCallback(early_stopping_patience=patience)]
    except Exception as e:
        print("[Compat] EarlyStoppingCallback unavailable:", e)
        return []


Transformers version loaded in memory: 4.44.2
Sample of supported TrainingArguments kwargs: ['accelerator_config', 'adafactor', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'auto_find_batch_size', 'batch_eval_metrics', 'bf16', 'bf16_full_eval', 'data_seed', 'dataloader_drop_last', 'dataloader_num_workers'] ...


## SECTION 2

In [3]:

# ============================================================================
# SECTION 2: IMPORTS AND BASIC SETUP
# ============================================================================

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


In [4]:
import os, random, json, math
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

from transformers import (
    AutoTokenizer, AutoModel, TrainingArguments, Trainer,
    DataCollatorWithPadding, EarlyStoppingCallback
)

def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

## SECTION 3

In [18]:
# SECTION 3

data_path = '/content/adjudications_2025-10-21.csv'
CSV_PATH = '/content/adjudications_2025-10-21.csv'

# Your exact column names
TITLE_COL = "Title"
TEXT_COL  = "Comment"
SENT_COL  = "Final Sentiment"
POL_COL   = "Final Polarization"

MODEL_CONFIGS = {
    "mbert":       {"name": "bert-base-multilingual-cased", "desc": "mBERT (104 langs)"},
    "xlm_roberta": {"name": "xlm-roberta-base",             "desc": "XLM-R base"},
    "rembert":     {"name": "google/rembert",               "desc": "RemBERT"},
}
MODELS_TO_RUN = ["mbert", "xlm_roberta"]

# Training hyperparams
MAX_LENGTH = 224
EPOCHS = 4
BATCH_SIZE = 16
LR = 2e-5
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.06
EARLY_STOP_PATIENCE = 2

# --- Per-task loss config (these worked well) ---
USE_FOCAL_SENTIMENT = False
USE_FOCAL_POLARITY  = True
FOCAL_GAMMA_POLARITY = 1.2
LABEL_SMOOTH_SENTIMENT = 0.05

# Task loss weighting (keep polarity gentle)
TASK_LOSS_WEIGHTS = {"sentiment": 1.0, "polarization": 1.05}

# Per-class multipliers (on top of balanced weights from train split)
CLASS_WEIGHT_MULT = {
    "sentiment": {
        "negative": 1.40,
        "neutral":  1.00,
        "positive": 1.10
    },
    "polarization": {
        "non_polarized": 1.00,
        "objective":     2.00,
        "partisan":      1.00
    }
}

# Oversampling (blended joint)
USE_OVERSAMPLING = True
USE_JOINT_OVERSAMPLING = True
JOINT_ALPHA = 0.40
JOINT_OVERSAMPLING_MAX_MULT = 3.0

# --- NEW: small MLP head before classifiers ---
HEAD_HIDDEN = 256
HEAD_DROPOUT = 0.15

# Output dir
OUT_DIR = "./runs_multitask"
os.makedirs(OUT_DIR, exist_ok=True)



## SECTION 4

In [6]:
# ===== Section 4 — Load & Prepare Data (updated for multipliers) =====
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()

required = [TITLE_COL, TEXT_COL, SENT_COL, POL_COL]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}. Found: {list(df.columns)}")

df = df.dropna(subset=[TITLE_COL, TEXT_COL, SENT_COL, POL_COL]).reset_index(drop=True)

# Encode labels
from sklearn.preprocessing import LabelEncoder
sent_le = LabelEncoder().fit(df[SENT_COL])
pol_le  = LabelEncoder().fit(df[POL_COL])

df["sent_y"] = sent_le.transform(df[SENT_COL])
df["pol_y"]  = pol_le.transform(df[POL_COL])

num_sent_classes = len(sent_le.classes_)
num_pol_classes  = len(pol_le.classes_)

print("Sentiment classes:", dict(enumerate(sent_le.classes_)))
print("Polarization classes:", dict(enumerate(pol_le.classes_)))

# Splits (stratify by sentiment)
from sklearn.model_selection import train_test_split
X = df[[TITLE_COL, TEXT_COL]].copy()
y_sent = df["sent_y"].values
y_pol  = df["pol_y"].values

X_train, X_tmp, ysent_train, ysent_tmp, ypol_train, ypol_tmp = train_test_split(
    X, y_sent, y_pol, test_size=0.3, random_state=42, stratify=y_sent
)
X_val, X_test, ysent_val, ysent_test, ypol_val, ypol_test = train_test_split(
    X_tmp, ysent_tmp, ypol_tmp, test_size=0.5, random_state=42, stratify=ysent_tmp
)

print("Train size:", len(X_train), "Val size:", len(X_val), "Test size:", len(X_test))

# Balanced class weights from TRAIN only
from sklearn.utils.class_weight import compute_class_weight
import numpy as np, json, os

def safe_class_weights(y, n_classes):
    classes = np.arange(n_classes)
    counts = np.bincount(y, minlength=n_classes)
    if np.any(counts == 0):
        return np.ones(n_classes, dtype=np.float32)
    return compute_class_weight("balanced", classes=classes, y=y).astype(np.float32)

sent_weights_np = safe_class_weights(ysent_train, num_sent_classes)
pol_weights_np  = safe_class_weights(ypol_train,  num_pol_classes)

# Apply user multipliers by class name
sent_name_to_idx = {name: i for i, name in enumerate(sent_le.classes_)}
pol_name_to_idx  = {name: i for i, name in enumerate(pol_le.classes_)}

for cname, mult in CLASS_WEIGHT_MULT["sentiment"].items():
    if cname in sent_name_to_idx:
        sent_weights_np[sent_name_to_idx[cname]] *= float(mult)

for cname, mult in CLASS_WEIGHT_MULT["polarization"].items():
    if cname in pol_name_to_idx:
        pol_weights_np[pol_name_to_idx[cname]] *= float(mult)

print("Final sentiment class weights:", {sent_le.classes_[i]: float(w) for i, w in enumerate(sent_weights_np)})
print("Final polarization class weights:", {pol_le.classes_[i]: float(w) for i, w in enumerate(pol_weights_np)})

# Save label maps
with open(os.path.join(OUT_DIR, "label_map_sentiment.json"), "w") as f:
    json.dump({int(k): v for k, v in dict(enumerate(sent_le.classes_)).items()}, f, indent=2)
with open(os.path.join(OUT_DIR, "label_map_polarization.json"), "w") as f:
    json.dump({int(k): v for k, v in dict(enumerate(pol_le.classes_)).items()}, f, indent=2)


Sentiment classes: {0: 'negative', 1: 'neutral', 2: 'positive'}
Polarization classes: {0: 'non_polarized', 1: 'objective', 2: 'partisan'}
Train size: 5718 Val size: 1225 Test size: 1226
Final sentiment class weights: {'negative': 0.8086060285568237, 'neutral': 1.1860610246658325, 'positive': 2.5852034091949463}
Final polarization class weights: {'non_polarized': 1.1081395149230957, 'objective': 10.89142894744873, 'partisan': 0.5224780440330505}


## SECTION 5

In [7]:
# ===== Section 5 — Dataset & Collator (proper text-pair encoding) =====
from torch.utils.data import Dataset

class TaglishDataset(Dataset):
    def __init__(self, titles, texts, y_sent, y_pol, tokenizer, max_length=224):
        self.titles = list(titles)
        self.texts  = list(texts)
        self.y_sent = np.array(y_sent)
        self.y_pol  = np.array(y_pol)
        self.tok = tokenizer
        self.max_length = max_length
        # mBERT has token_type_ids; XLM-R/RemBERT don't, and that's fine.
        self.use_token_type = "token_type_ids" in tokenizer.model_input_names

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Pass title as text, comment as text_pair so the tokenizer inserts the correct separators.
        # We also bias truncation to the comment since titles are short.
        enc = self.tok(
            text=str(self.titles[idx]),
            text_pair=str(self.texts[idx]),
            truncation="only_second",     # keep the title intact; trim the comment if needed
            max_length=self.max_length,
            return_token_type_ids=self.use_token_type,
        )
        item = {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "sentiment_labels": torch.tensor(self.y_sent[idx], dtype=torch.long),
            "polarization_labels": torch.tensor(self.y_pol[idx], dtype=torch.long),
        }
        if self.use_token_type and "token_type_ids" in enc:
            item["token_type_ids"] = enc["token_type_ids"]
        return item


## SECTION 6

In [19]:
# ===== Section 6 — Multi-Task Model (pooling + MLP heads) =====
import torch
import torch.nn as nn
from transformers import AutoModel

def mean_pooling(token_embeddings, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    summed = (token_embeddings * mask).sum(dim=1)
    denom = mask.sum(dim=1).clamp(min=1e-9)
    return summed / denom

class MultiTaskModel(nn.Module):
    def __init__(self, base_model_name: str, num_sent: int, num_pol: int, dropout: float = 0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(base_model_name)
        self.hidden = self.encoder.config.hidden_size

        self.trunk = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.hidden, HEAD_HIDDEN),
            nn.GELU(),
            nn.LayerNorm(HEAD_HIDDEN),
            nn.Dropout(HEAD_DROPOUT),
        )
        self.head_sent = nn.Linear(HEAD_HIDDEN, num_sent)
        self.head_pol  = nn.Linear(HEAD_HIDDEN, num_pol)

    def _pool(self, outputs, attention_mask):
        # Flexible representation pooling
        if REP_POOLING == "pooler" and hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            return outputs.pooler_output
        if REP_POOLING == "cls":
            return outputs.last_hidden_state[:, 0]
        # default: last4_mean
        hs = outputs.hidden_states  # tuple of [layer0..last]
        last4 = torch.stack(hs[-4:]).mean(dim=0)       # [B, T, H]
        return mean_pooling(last4, attention_mask)     # [B, H]

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                sentiment_labels=None,
                polarization_labels=None):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids if token_type_ids is not None else None,
            output_hidden_states=(REP_POOLING != "pooler")  # needed for last4_mean/cls
        )
        pooled = self._pool(outputs, attention_mask)
        z = self.trunk(pooled)
        return {"logits": (self.head_sent(z), self.head_pol(z))}


## SECTION 7

In [9]:
# SECTION 7

def compute_metrics_multi(eval_pred):
    (sent_logits, pol_logits) = eval_pred.predictions
    (y_sent, y_pol) = eval_pred.label_ids

    ps = np.argmax(sent_logits, axis=1)
    pp = np.argmax(pol_logits, axis=1)

    # Macro metrics
    sent_report = classification_report(y_sent, ps, output_dict=True, zero_division=0)
    pol_report  = classification_report(y_pol,  pp, output_dict=True, zero_division=0)

    sent_f1 = sent_report["macro avg"]["f1-score"]
    pol_f1  = pol_report["macro avg"]["f1-score"]
    macro_f1_avg = (sent_f1 + pol_f1) / 2.0

    return {
        "sent_acc": sent_report["accuracy"],
        "sent_prec": sent_report["macro avg"]["precision"],
        "sent_rec": sent_report["macro avg"]["recall"],
        "sent_f1": sent_f1,

        "pol_acc": pol_report["accuracy"],
        "pol_prec": pol_report["macro avg"]["precision"],
        "pol_rec": pol_report["macro avg"]["recall"],
        "pol_f1": pol_f1,

        "macro_f1_avg": macro_f1_avg
    }


## SECTION 8

In [20]:
# ===== Section 8 — Custom Trainer (R-Drop + LLRD + safe prediction_step) =====
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
from torch.utils.data import DataLoader
from torch.optim import AdamW

class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, logits, target):
        logp = F.log_softmax(logits, dim=1)
        p = torch.exp(logp)
        loss = F.nll_loss((1 - p) ** self.gamma * logp, target, weight=self.weight, reduction="none")
        return loss.mean() if self.reduction == "mean" else loss.sum()

def _sym_kl_with_logits(logits1, logits2):
    p = F.log_softmax(logits1, dim=-1);  q = F.log_softmax(logits2, dim=-1)
    p_exp, q_exp = p.exp(), q.exp()
    return 0.5 * (F.kl_div(p, q_exp, reduction="batchmean") + F.kl_div(q, p_exp, reduction="batchmean"))

class MultiTaskTrainer(Trainer):
    def __init__(self, *args, class_weights=None, task_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights or {}
        self.task_weights  = task_weights or {"sentiment": 1.0, "polarization": 1.0}
        self._custom_train_sampler = None

    # ----- LLRD optimizer -----
    def create_optimizer(self):
        if self.optimizer is not None:
            return self.optimizer
        if not USE_LLRD:
            self.optimizer = AdamW(self.get_decay_parameter_groups(self.model), lr=LR, weight_decay=WEIGHT_DECAY)
            return self.optimizer

        no_decay = ["bias", "LayerNorm.weight", "LayerNorm.bias"]
        encoder = self.model.encoder
        n_layers = getattr(encoder.config, "num_hidden_layers", 12)
        # Try to access sequential layers
        layers = getattr(getattr(encoder, "encoder", encoder), "layer", None)
        if layers is None:
            # Fallback: no LLRD if we can't find layers
            self.optimizer = AdamW(self.get_decay_parameter_groups(self.model), lr=LR, weight_decay=WEIGHT_DECAY)
            return self.optimizer

        param_groups = []

        # Embeddings (lowest lr)
        emb = getattr(encoder, "embeddings", None)
        if emb is not None:
            lr_emb = LR * (LLRD_DECAY ** n_layers)
            decay, nodecay = [], []
            for n, p in emb.named_parameters():
                (nodecay if any(nd in n for nd in no_decay) else decay).append(p)
            if decay:   param_groups.append({"params": decay,   "lr": lr_emb, "weight_decay": WEIGHT_DECAY})
            if nodecay: param_groups.append({"params": nodecay, "lr": lr_emb, "weight_decay": 0.0})

        # Encoder blocks (increasing LR toward the top)
        for i in range(n_layers):
            block = layers[i]
            lr_i = LR * (LLRD_DECAY ** (n_layers - 1 - i))
            decay, nodecay = [], []
            for n, p in block.named_parameters():
                (nodecay if any(nd in n for nd in no_decay) else decay).append(p)
            if decay:   param_groups.append({"params": decay,   "lr": lr_i, "weight_decay": WEIGHT_DECAY})
            if nodecay: param_groups.append({"params": nodecay, "lr": lr_i, "weight_decay": 0.0})

        # Pooler (if any)
        pooler = getattr(encoder, "pooler", None)
        if pooler is not None:
            decay, nodecay = [], []
            for n, p in pooler.named_parameters():
                (nodecay if any(nd in n for nd in no_decay) else decay).append(p)
            if decay:   param_groups.append({"params": decay,   "lr": LR, "weight_decay": WEIGHT_DECAY})
            if nodecay: param_groups.append({"params": nodecay, "lr": LR, "weight_decay": 0.0})

        # Heads/trunk (highest LR)
        head_lr = LR * HEAD_LR_MULT
        head_modules = [self.model.trunk, self.model.head_sent, self.model.head_pol]
        decay, nodecay = [], []
        for m in head_modules:
            for n, p in m.named_parameters():
                (nodecay if any(nd in n for nd in no_decay) else decay).append(p)
        if decay:   param_groups.append({"params": decay,   "lr": head_lr, "weight_decay": WEIGHT_DECAY})
        if nodecay: param_groups.append({"params": nodecay, "lr": head_lr, "weight_decay": 0.0})

        self.optimizer = AdamW(param_groups, lr=LR)  # lr here is ignored per-group
        return self.optimizer

    def set_train_sampler(self, sampler):
        self._custom_train_sampler = sampler

    def get_train_dataloader(self):
        if self.train_dataset is None:
            return None
        if self._custom_train_sampler is not None:
            return DataLoader(
                self.train_dataset,
                batch_size=self.args.train_batch_size,
                sampler=self._custom_train_sampler,
                collate_fn=self.data_collator,
                drop_last=self.args.dataloader_drop_last,
                num_workers=self.args.dataloader_num_workers,
                pin_memory=self.args.dataloader_pin_memory,
            )
        return super().get_train_dataloader()

    def _sent_loss_fn(self, weight, logits, target):
        return nn.CrossEntropyLoss(weight=weight, label_smoothing=float(LABEL_SMOOTH_SENTIMENT))(logits, target)
    def _pol_loss_fn(self, weight, logits, target):
        return FocalLoss(weight=weight, gamma=FOCAL_GAMMA_POLARITY)(logits, target)

    def compute_loss(self, model, inputs, return_outputs=False):
        y_sent = inputs.pop("sentiment_labels")
        y_pol  = inputs.pop("polarization_labels")
        # R-Drop: two forward passes with dropout
        if USE_RDROP and model.training:
            outputs1 = model(**inputs)
            outputs2 = model(**inputs)
            s1, p1 = outputs1["logits"]
            s2, p2 = outputs2["logits"]

            ws = self.class_weights.get("sentiment", None); ws = ws.to(s1.device) if ws is not None else None
            wp = self.class_weights.get("polarization", None); wp = wp.to(p1.device) if wp is not None else None

            ce_s = 0.5 * (self._sent_loss_fn(ws, s1, y_sent) + self._sent_loss_fn(ws, s2, y_sent))
            ce_p = 0.5 * (self._pol_loss_fn(wp,  p1, y_pol)  + self._pol_loss_fn(wp,  p2, y_pol))
            kl_s = _sym_kl_with_logits(s1, s2)
            kl_p = _sym_kl_with_logits(p1, p2)

            w_s = float(self.task_weights.get("sentiment", 1.0))
            w_p = float(self.task_weights.get("polarization", 1.0))
            loss = w_s * ce_s + w_p * ce_p + RDROP_ALPHA * (kl_s + kl_p)
            if return_outputs:
                return loss, {"logits": (s1, p1)}
            return loss

        # Standard single forward
        outputs = model(**inputs)
        s, p = outputs["logits"]

        ws = self.class_weights.get("sentiment", None); ws = ws.to(s.device) if ws is not None else None
        wp = self.class_weights.get("polarization", None); wp = wp.to(p.device) if wp is not None else None

        loss_s = self._sent_loss_fn(ws, s, y_sent) if not USE_FOCAL_SENTIMENT else FocalLoss(weight=ws, gamma=1.5)(s, y_sent)
        loss_p = self._pol_loss_fn(wp,  p, y_pol)  if USE_FOCAL_POLARITY else nn.CrossEntropyLoss(weight=wp)(p, y_pol)

        w_s = float(self.task_weights.get("sentiment", 1.0))
        w_p = float(self.task_weights.get("polarization", 1.0))
        loss = w_s * loss_s + w_p * loss_p

        if return_outputs:
            outputs = dict(outputs); outputs["labels"] = (y_sent, y_pol)
            return loss, outputs
        return loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        # Safe for inference (no labels provided)
        y_sent = inputs.get("sentiment_labels", None)
        y_pol  = inputs.get("polarization_labels", None)

        model_inputs = {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]}
        if "token_type_ids" in inputs:
            model_inputs["token_type_ids"] = inputs["token_type_ids"]

        model.eval()
        with torch.no_grad():
            outputs = model(**model_inputs)
            s, p = outputs["logits"]

        loss = None
        logits = (s.detach(), p.detach())
        labels = (y_sent, y_pol) if isinstance(y_sent, torch.Tensor) and isinstance(y_pol, torch.Tensor) else None
        return (loss, logits, labels)


## SECTION 9

In [21]:
# ===== Section 9 — Train/Evaluate One Model =====
def train_eval_one_model(model_key: str,
                         X_tr: pd.DataFrame, X_v: pd.DataFrame, X_te: pd.DataFrame,
                         ysent_tr: np.ndarray, ysent_v: np.ndarray, ysent_te: np.ndarray,
                         ypol_tr: np.ndarray,  ypol_v: np.ndarray,  ypol_te: np.ndarray,
                         sent_w_np: np.ndarray, pol_w_np: np.ndarray):
    base_name = MODEL_CONFIGS[model_key]["name"]
    run_dir = os.path.join(OUT_DIR, f"{model_key}")
    os.makedirs(run_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(base_name)
    collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

    # Build datasets
    tr_titles, tr_texts = X_tr[TITLE_COL].values, X_tr[TEXT_COL].values
    v_titles,  v_texts  = X_v[TITLE_COL].values,  X_v[TEXT_COL].values
    te_titles, te_texts = X_te[TITLE_COL].values, X_te[TEXT_COL].values

    train_ds = TaglishDataset(tr_titles, tr_texts, ysent_tr, ypol_tr, tokenizer, max_length=MAX_LENGTH)
    val_ds   = TaglishDataset(v_titles,  v_texts,  ysent_v,  ypol_v,  tokenizer, max_length=MAX_LENGTH)
    test_ds  = TaglishDataset(te_titles, te_texts, ysent_te, ypol_te, tokenizer, max_length=MAX_LENGTH)

    # Model
    model = MultiTaskModel(base_name, num_sent_classes, num_pol_classes).to(device)

    # Class weights (torch tensors)
    sent_w = torch.tensor(sent_w_np, dtype=torch.float32)
    pol_w  = torch.tensor(pol_w_np,  dtype=torch.float32)

    args = TrainingArguments(
        output_dir=run_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LR,
        weight_decay=WEIGHT_DECAY,
        warmup_ratio=WARMUP_RATIO,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1_avg",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        logging_dir=os.path.join(run_dir, "logs"),
        logging_steps=50,
        report_to="none",
        seed=42,
        # Safety tweaks so our custom inputs aren't dropped and eval is smooth
        remove_unused_columns=False,
        eval_accumulation_steps=1
    )

    trainer = MultiTaskTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_multi,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOP_PATIENCE)],
        class_weights={"sentiment": sent_w, "polarization": pol_w}
    )

    trainer.train()

    # Eval on test
    test_out = trainer.predict(test_ds)
    metrics = {f"test_{k}": float(v) for k, v in test_out.metrics.items()}

    # Save artifacts
    trainer.save_model(run_dir)
    tokenizer.save_pretrained(run_dir)
    with open(os.path.join(run_dir, "metrics_test.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    # Confusion matrices
    sent_logits, pol_logits = test_out.predictions
    ysent_pred = np.argmax(sent_logits, axis=1)
    ypol_pred  = np.argmax(pol_logits,  axis=1)

    cm_sent = confusion_matrix(ysent_te, ysent_pred, labels=list(range(num_sent_classes)))
    cm_pol  = confusion_matrix(ypol_te,  ypol_pred,  labels=list(range(num_pol_classes)))

    np.save(os.path.join(run_dir, "cm_sent.npy"), cm_sent)
    np.save(os.path.join(run_dir, "cm_pol.npy"),  cm_pol)

    # Quick plots
    def plot_cm(cm, labels, title, path_png):
        fig, ax = plt.subplots(figsize=(4.5, 4))
        im = ax.imshow(cm, interpolation="nearest")
        ax.set_title(title)
        ax.set_xlabel("Predicted")
        ax.set_ylabel("True")
        ax.set_xticks(range(len(labels))); ax.set_xticklabels(labels, rotation=45, ha="right")
        ax.set_yticks(range(len(labels))); ax.set_yticklabels(labels)
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, cm[i, j], ha="center", va="center")
        fig.colorbar(im, ax=ax, fraction=0.046)
        plt.tight_layout()
        plt.savefig(path_png, dpi=160)
        plt.close(fig)

    plot_cm(cm_sent, sent_le.classes_, "Sentiment Confusion", os.path.join(run_dir, "cm_sent.png"))
    plot_cm(cm_pol,  pol_le.classes_,  "Polarization Confusion", os.path.join(run_dir, "cm_pol.png"))

    # Text reports
    rep_sent = classification_report(ysent_te, ysent_pred, target_names=sent_le.classes_, digits=4, zero_division=0)
    rep_pol  = classification_report(ypol_te,  ypol_pred,  target_names=pol_le.classes_,  digits=4, zero_division=0)
    with open(os.path.join(run_dir, "report_sentiment.txt"), "w") as f:
        f.write(rep_sent)
    with open(os.path.join(run_dir, "report_polarization.txt"), "w") as f:
        f.write(rep_pol)

    summary_row = {
        "model_key": model_key,
        "base_name": base_name,
        **metrics
    }
    return summary_row, (ysent_pred, ypol_pred)


## SECTION 10


In [12]:
# SECTION 10

results = []
pred_cache = {}

for key in MODELS_TO_RUN:
    print(f"\n=== Running {key} -> {MODEL_CONFIGS[key]['name']} ===")
    row, preds = train_eval_one_model(
        key,
        X_train, X_val, X_test,
        ysent_train, ysent_val, ysent_test,
        ypol_train,  ypol_val,  ypol_test,
        sent_weights_np, pol_weights_np
    )
    results.append(row)
    pred_cache[key] = preds

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(OUT_DIR, "summary_results.csv"), index=False)
results_df



=== Running mbert -> bert-base-multilingual-cased ===


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Sent Acc,Sent Prec,Sent Rec,Sent F1,Pol Acc,Pol Prec,Pol Rec,Pol F1,Macro F1 Avg
1,1.900700,No log,0.631020,0.565780,0.531325,0.469748,0.637551,0.569507,0.611492,0.496212,0.482980
2,1.443100,No log,0.644898,0.611655,0.654841,0.626530,0.681633,0.568422,0.672154,0.585824,0.606177
3,1.289500,No log,0.656327,0.615545,0.680473,0.635534,0.684082,0.571851,0.657221,0.588777,0.612155
4,1.033500,No log,0.664490,0.616892,0.672397,0.634183,0.706122,0.588742,0.661937,0.608741,0.621462


,model_key,base_name,test_test_sent_acc,test_test_sent_prec,test_test_sent_rec,test_test_sent_f1,test_test_pol_acc,test_test_pol_prec,test_test_pol_rec,test_test_pol_f1,test_test_macro_f1_avg,test_test_runtime,test_test_samples_per_second,test_test_steps_per_second
0,mbert,bert-base-multilingual-cased,0.671289,0.634502,0.674686,0.650642,0.713703,0.592254,0.64263,0.607607,0.629124,4.2179,290.665,18.255
1,xlm_roberta,xlm-roberta-base,0.690049,0.641377,0.692149,0.657848,0.705546,0.583282,0.65654,0.603624,0.630736,5.1490,238.105,14.954


### SECTION 10A


In [13]:
# ============================================================================
# SECTION 10A — VERIFY ARTIFACTS & RESOLVE TOKENIZER + WEIGHTS (v2)
# Builds maps for: tokenizer_dir (usually run root) and weights_dir (checkpoint or run root).
# Run AFTER Section 10 (training) and BEFORE 11B/11C.
# ============================================================================

import os, re, json
from typing import Optional, Dict

def _has_weights(path: str) -> bool:
    return os.path.isfile(os.path.join(path, "pytorch_model.bin")) or os.path.isfile(os.path.join(path, "model.safetensors"))

def _has_tokenizer(path: str) -> bool:
    # Minimal tokenizer files
    return (
        os.path.isfile(os.path.join(path, "tokenizer.json")) or
        os.path.isfile(os.path.join(path, "vocab.txt")) or
        os.path.isfile(os.path.join(path, "spiece.model"))
    )

def _list_checkpoints(run_dir: str):
    if not os.path.isdir(run_dir): return []
    chks = []
    for name in os.listdir(run_dir):
        p = os.path.join(run_dir, name)
        if os.path.isdir(p) and re.match(r"^checkpoint-\d+$", name):
            chks.append(p)
    # sort by gl


## SECTION 11


In [14]:
# ===== Section 11 — Detailed Breakdown Reports (per-class + cross-slices) =====
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import os
import json

def per_class_breakdown(y_true, y_pred, class_names):
    rep = classification_report(
        y_true, y_pred,
        target_names=list(class_names),
        output_dict=True, zero_division=0
    )
    # Keep only the class rows in the given order
    rows = []
    for cname in class_names:
        if cname in rep:
            rows.append({
                "class": cname,
                "precision": rep[cname]["precision"],
                "recall":    rep[cname]["recall"],
                "f1":        rep[cname]["f1-score"],
                "support":   int(rep[cname]["support"]),
            })
        else:
            rows.append({"class": cname, "precision": 0.0, "recall": 0.0, "f1": 0.0, "support": 0})
    return pd.DataFrame(rows)

def cross_slice_breakdown(
    slice_true,  # array of ints for the slicing label (e.g., true sentiment indices)
    slice_names, # names of the slicing label classes (e.g., sentiment class names)
    task_true,   # array of ints for the task we evaluate (e.g., true polarity indices)
    task_pred,   # array of ints for the task predictions (e.g., predicted polarity indices)
    task_names,  # names of the task classes (e.g., polarity class names)
    slice_label  # string for the slice axis name, e.g., "sentiment" or "polarity"
):
    """
    For each class s in slice_true, evaluate the task predictions on the subset where slice_true == s.
    Returns one row per slice value, including macro-F1, accuracy, and per-class F1 for the task.
    """
    rows = []
    for idx, sname in enumerate(slice_names):
        mask = (slice_true == idx)
        n = int(mask.sum())
        if n == 0:
            # No samples for this slice in test set
            row = {"slice": sname, "support": 0, "accuracy": np.nan, "macro_f1": np.nan}
            for tname in task_names:
                row[f"f1_{tname}"] = np.nan
            rows.append(row)
            continue

        rep = classification_report(
            task_true[mask], task_pred[mask],
            target_names=list(task_names),
            output_dict=True, zero_division=0
        )
        row = {
            "slice": sname,
            "support": n,
            "accuracy": rep["accuracy"],
            "macro_f1": rep["macro avg"]["f1-score"],
        }
        for tname in task_names:
            row[f"f1_{tname}"] = rep[tname]["f1-score"]
        rows.append(row)

    df = pd.DataFrame(rows)
    # Sort slices by support (desc) for readability
    df = df.sort_values(by="support", ascending=False).reset_index(drop=True)
    return df

# Where to save things
DETAILS_DIR = os.path.join(OUT_DIR, "details")
os.makedirs(DETAILS_DIR, exist_ok=True)

all_breakdowns = {}

for key in MODELS_TO_RUN:
    print(f"\n=== Detailed breakdowns for {key} ===")
    ysent_pred, ypol_pred = pred_cache[key]

    # ---- Per-class reports on the full test set
    sent_per_class = per_class_breakdown(ysent_test, ysent_pred, sent_le.classes_)
    pol_per_class  = per_class_breakdown(ypol_test,  ypol_pred,  pol_le.classes_)

    # Save + show
    sent_csv = os.path.join(DETAILS_DIR, f"{key}_sentiment_per_class.csv")
    pol_csv  = os.path.join(DETAILS_DIR, f"{key}_polarization_per_class.csv")
    sent_per_class.to_csv(sent_csv, index=False)
    pol_per_class.to_csv(pol_csv, index=False)

    print("\nSentiment — per class (precision/recall/F1/support):")
    display(sent_per_class)

    print("\nPolarization — per class (precision/recall/F1/support):")
    display(pol_per_class)

    # ---- Cross-slice reports
    # Polarity performance within each (true) sentiment slice
    pol_given_sent = cross_slice_breakdown(
        slice_true=ysent_test, slice_names=sent_le.classes_,
        task_true=ypol_test,   task_pred=ypol_pred, task_names=pol_le.classes_,
        slice_label="sentiment"
    )
    pol_given_sent_csv = os.path.join(DETAILS_DIR, f"{key}_polarity_given_sentiment.csv")
    pol_given_sent.to_csv(pol_given_sent_csv, index=False)

    print("\nPolarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):")
    display(pol_given_sent)

    # Sentiment performance within each (true) polarity slice
    sent_given_pol = cross_slice_breakdown(
        slice_true=ypol_test,  slice_names=pol_le.classes_,
        task_true=ysent_test,  task_pred=ysent_pred, task_names=sent_le.classes_,
        slice_label="polarity"
    )
    sent_given_pol_csv = os.path.join(DETAILS_DIR, f"{key}_sentiment_given_polarity.csv")
    sent_given_pol.to_csv(sent_given_pol_csv, index=False)

    print("\nSentiment performance within each Polarity slice (accuracy / macro-F1 / per-class F1):")
    display(sent_given_pol)

    # Keep for a single JSON bundle if you like
    all_breakdowns[key] = {
        "sentiment_per_class_csv": sent_csv,
        "polarization_per_class_csv": pol_csv,
        "polarity_given_sentiment_csv": pol_given_sent_csv,
        "sentiment_given_polarity_csv": sent_given_pol_csv
    }

# Optional: write an index JSON pointing to all CSVs
with open(os.path.join(DETAILS_DIR, "index.json"), "w") as f:
    json.dump(all_breakdowns, f, indent=2)
print("\nSaved detailed breakdowns to:", DETAILS_DIR)



=== Detailed breakdowns for mbert ===

Sentiment — per class (precision/recall/F1/support):


,class,precision,recall,f1,support
0,negative,0.782946,0.713277,0.746489,708
1,neutral,0.506925,0.530435,0.518414,345
2,positive,0.613636,0.780347,0.687023,173



Polarization — per class (precision/recall/F1/support):


,class,precision,recall,f1,support
0,non_polarized,0.536036,0.727829,0.617380,327
1,objective,0.355769,0.468354,0.404372,79
2,partisan,0.884956,0.731707,0.801068,820



Polarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):


...


Polarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):


,slice,support,accuracy,macro_f1,f1_non_polarized,f1_objective,f1_partisan
0,negative,708,0.761299,0.553673,0.542955,0.264706,0.853359
1,neutral,345,0.643478,0.618392,0.676923,0.527273,0.650980
2,positive,173,0.601156,0.516687,0.589928,0.285714,0.674419



Sentiment performance within each Polarity slice (accuracy / macro-F1 / per-class F1):


,slice,support,accuracy,macro_f1,f1_negative,f1_neutral,f1_positive
0,partisan,820,0.732927,0.630158,0.821650,0.366534,0.702290
1,non_polarized,327,0.593272,0.598216,0.571429,0.591640,0.631579
2,objective,79,0.645570,0.646991,0.500000,0.682353,0.758621



Saved detailed breakdowns to: ./runs_multitask/details


### SECTION 11B


In [15]:
# ============================================================================
# SECTION 11B — SIMPLE POLARITY CALIBRATION (objective bias) — SELF-HEALING
# This cell will:
#   1) Rebuild tokenizer/weights maps if the JSONs are missing,
#   2) Load tokenizer from run root (or base model) and weights from checkpoint/run root,
#   3) Do objective-only bias calibration on VAL and report TEST before/after.
# Prereqs: OUT_DIR, MODELS_TO_RUN, MODEL_CONFIGS, MultiTaskModel, MultiTaskTrainer,
#          X_val, X_test, ypol_val, ypol_test, pol_le, TITLE_COL, TEXT_COL, MAX_LENGTH, device.
# ============================================================================

import os, re, json, numpy as np, torch
from typing import Optional, Dict
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset

# ---------- Helpers to (re)build the tokenizer/weights maps ----------
def _has_weights(path: str) -> bool:
    return os.path.isfile(os.path.join(path, "pytorch_model.bin")) or os.path.isfile(os.path.join(path, "model.safetensors"))

def _has_tokenizer(path: str) -> bool:
    return (
        os.path.isfile(os.path.join(path, "tokenizer.json")) or
        os.path.isfile(os.path.join(path, "vocab.txt")) or
        os.path.isfile(os.path.join(path, "spiece.model"))
    )

def _list_checkpoints(run_dir: str):
    if not os.path.isdir(run_dir): return []
    chks = []
    for name in os.listdir(run_dir):
        p = os.path.join(run_dir, name)
        if os.path.isdir(p) and re.match(r"^checkpoint-\d+$", name):
            chks.append(p)
    chks.sort(key=lambda p: int(os.path.basename(p).split("-")[-1]), reverse=True)
    return chks

def _rebuild_maps():
    tok_map: Dict[str, Optional[str]] = {}
    wts_map: Dict[str, Optional[str]] = {}
    print("=== (11B) Rebuilding tokenizer/weights maps ===")
    for key in MODELS_TO_RUN:
        run_root = os.path.join(OUT_DIR, key)
        tok_dir = run_root if _has_tokenizer(run_root) else None
        weights_dir = None
        for chk in _list_checkpoints(run_root):
            if _has_weights(chk):
                weights_dir = chk
                break
        if weights_dir is None and _has_weights(run_root):
            weights_dir = run_root

        tok_map[key] = tok_dir
        wts_map[key] = weights_dir
        print(f"[{key}] tokenizer_dir: {tok_dir or '(fallback to base model)'}")
        print(f"[{key}]   weights_dir: {weights_dir or '(NOT FOUND)'}")
        if not weights_dir:
            # list contents to help debug
            if os.path.isdir(run_root):
                print("  Contents of run dir:")
                for item in sorted(os.listdir(run_root)):
                    print("   -", item)
            else:
                print("  Run dir missing:", run_root)

    os.makedirs(OUT_DIR, exist_ok=True)
    with open(os.path.join(OUT_DIR, "tokenizer_map.json"), "w") as f:
        json.dump(tok_map, f, indent=2)
    with open(os.path.join(OUT_DIR, "weights_map.json"), "w") as f:
        json.dump(wts_map, f, indent=2)

    return tok_map, wts_map

# Load or rebuild maps
tok_map_path = os.path.join(OUT_DIR, "tokenizer_map.json")
wts_map_path = os.path.join(OUT_DIR, "weights_map.json")
if os.path.isfile(tok_map_path) and os.path.isfile(wts_map_path):
    TOKENIZER_MAP = json.load(open(tok_map_path))
    WEIGHTS_MAP   = json.load(open(wts_map_path))
else:
    TOKENIZER_MAP, WEIGHTS_MAP = _rebuild_maps()

# ---------- Data helpers ----------
class _PlainPairDS(Dataset):
    def __init__(self, titles, texts, tokenizer, max_length=224):
        self.titles, self.texts = list(titles), list(texts)
        self.tok = tokenizer
        self.max_length = max_length
        self.use_tt = "token_type_ids" in tokenizer.model_input_names
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        return self.tok(
            text=str(self.titles[idx]),
            text_pair=str(self.texts[idx]),
            truncation="only_second",
            max_length=self.max_length,
            return_token_type_ids=self.use_tt
        )

def _load_tok_and_model_for_key(model_key: str):
    # Tokenizer: prefer local run root; else base model name
    tok_dir = TOKENIZER_MAP.get(model_key) or MODEL_CONFIGS[model_key]["name"]
    tok = AutoTokenizer.from_pretrained(tok_dir)

    # Model: rebuild from base name; load weights from found dir
    base_name = MODEL_CONFIGS[model_key]["name"]
    model = MultiTaskModel(base_name, num_sent_classes, num_pol_classes)
    wdir = WEIGHTS_MAP.get(model_key)
    if not wdir:
        raise FileNotFoundError(
            f"[{model_key}] No weights dir found. Please re-run Section 10 (training). "
            f"Expected weights under {os.path.join(OUT_DIR, model_key)}"
        )
    bin_path = os.path.join(wdir, "pytorch_model.bin")
    safetensors_path = os.path.join(wdir, "model.safetensors")
    if os.path.isfile(bin_path):
        sd = torch.load(bin_path, map_location="cpu")
        model.load_state_dict(sd, strict=False)
    elif os.path.isfile(safetensors_path):
        from safetensors.torch import load_file as load_safetensors
        sd = load_safetensors(safetensors_path)
        model.load_state_dict(sd, strict=False)
    else:
        raise FileNotFoundError(f"[{model_key}] No weights file in {wdir} (need pytorch_model.bin or model.safetensors).")
    model.to(device)
    return tok, model

def _get_pol_logits(model_key, titles, texts):
    tok, model = _load_tok_and_model_for_key(model_key)
    collator = DataCollatorWithPadding(tokenizer=tok, padding=True)
    args = TrainingArguments(
        output_dir=os.path.join(OUT_DIR, model_key, "calib_tmp"),
        per_device_eval_batch_size=64,
        report_to="none"
    )
    dummy_trainer = MultiTaskTrainer(model=model, args=args, data_collator=collator, class_weights=None, task_weights=None)
    ds = _PlainPairDS(titles, texts, tok, MAX_LENGTH)
    out = dummy_trainer.predict(ds)
    _, pol_logits = out.predictions
    return pol_logits

def _apply_bias(pol_logits, bias_vec):
    return pol_logits + bias_vec.reshape(1, -1)

def _report(y_true, y_pred, title=None):
    rep = classification_report(y_true, y_pred, target_names=pol_le.classes_, output_dict=True, zero_division=0)
    if title:
        print(title)
    print(f"macro-F1 = {rep['macro avg']['f1-score']:.4f} | accuracy = {rep['accuracy']:.4f}")
    for cname in pol_le.classes_:
        r = rep[cname]
        print(f"  {cname:>13}: P={r['precision']:.3f} R={r['recall']:.3f} F1={r['f1-score']:.3f} (n={int(r['support'])})")
    return rep

# ---------- Run calibration ----------
CALIB_DIR = os.path.join(OUT_DIR, "calibration_simple")
os.makedirs(CALIB_DIR, exist_ok=True)

for key in MODELS_TO_RUN:
    print(f"\n=== Simple calibration for {key} ===")

    # Get polarity logits
    pol_val_logits = _get_pol_logits(key, X_val[TITLE_COL].values,  X_val[TEXT_COL].values)
    pol_tst_logits = _get_pol_logits(key, X_test[TITLE_COL].values, X_test[TEXT_COL].values)

    y_val = ypol_val
    y_tst = ypol_test

    # "objective" class index
    obj_idx = int(np.where(pol_le.classes_ == "objective")[0][0]) if "objective" in pol_le.classes_ else 1

    # Grid search bias on VAL
    biases = np.linspace(-1.0, 1.0, 41)
    best_b, best_macro = 0.0, -1.0
    for b in biases:
        bias_vec = np.zeros(pol_val_logits.shape[1], dtype=np.float32)
        bias_vec[obj_idx] = b
        y_pred = np.argmax(_apply_bias(pol_val_logits, bias_vec), axis=1)
        rep = classification_report(y_val, y_pred, target_names=pol_le.classes_, output_dict=True, zero_division=0)
        macro = rep["macro avg"]["f1-score"]
        if macro > best_macro:
            best_macro, best_b = macro, b

    print(f"Chosen bias for 'objective' on VAL: {best_b:+.2f} (macro-F1={best_macro:.3f})")

    # TEST before/after
    y_before = np.argmax(pol_tst_logits, axis=1)
    _ = _report(y_tst, y_before, title="TEST before bias:")
    bias_vec = np.zeros(pol_tst_logits.shape[1], dtype=np.float32)
    bias_vec[obj_idx] = best_b
    y_after = np.argmax(_apply_bias(pol_tst_logits, bias_vec), axis=1)
    rep_after = _report(y_tst, y_after, title="TEST after bias:")

    with open(os.path.join(CALIB_DIR, f"{key}_objective_bias.json"), "w") as f:
        json.dump({"chosen_bias": float(best_b), "test_macro_f1_after": float(rep_after["macro avg"]["f1-score"])}, f, indent=2)


=== (11B) Rebuilding tokenizer/weights maps ===
[mbert] tokenizer_dir: ./runs_multitask/mbert
[mbert]   weights_dir: ./runs_multitask/mbert/checkpoint-1432
[xlm_roberta] tokenizer_dir: ./runs_multitask/xlm_roberta
[xlm_roberta]   weights_dir: ./runs_multitask/xlm_roberta/checkpoint-1432

=== Simple calibration for mbert ===


Chosen bias for 'objective' on VAL: -1.00 (macro-F1=0.600)
TEST before bias:
macro-F1 = 0.5945 | accuracy = 0.7047
  non_polarized: P=0.538 R=0.713 F1=0.613 (n=327)
      objective: P=0.298 R=0.494 F1=0.371 (n=79)
       partisan: P=0.894 R=0.722 F1=0.799 (n=820)
TEST after bias:
macro-F1 = 0.6018 | accuracy = 0.7235
  non_polarized: P=0.538 R=0.761 F1=0.630 (n=327)
      objective: P=0.362 R=0.367 F1=0.365 (n=79)
       partisan: P=0.892 R=0.743 F1=0.810 (n=820)

=== Simple calibration for xlm_roberta ===


Chosen bias for 'objective' on VAL: -0.55 (macro-F1=0.623)
TEST before bias:
macro-F1 = 0.6036 | accuracy = 0.7055
  non_polarized: P=0.537 R=0.703 F1=0.609 (n=327)
      objective: P=0.321 R=0.544 F1=0.404 (n=79)
       partisan: P=0.892 R=0.722 F1=0.798 (n=820)
TEST after bias:
macro-F1 = 0.6110 | accuracy = 0.7162
  non_polarized: P=0.538 R=0.737 F1=0.622 (n=327)
      objective: P=0.363 R=0.468 F1=0.409 (n=79)
       partisan: P=0.888 R=0.732 F1=0.802 (n=820)


In [16]:
# ============================================================================
# SECTION 11C — MULTICLASS POLARITY CALIBRATION (v2)
# ============================================================================

from sklearn.metrics import classification_report
import numpy as np, json, os

def coord_search_biases(pol_logits_val, y_val, class_names, passes=2, grid=(-0.8, 0.8, 0.1)):
    lo, hi, step = grid
    C = pol_logits_val.shape[1]
    b = np.zeros(C, dtype=np.float32)

    def macro_f1_with(bias_vec):
        y_pred = np.argmax(pol_logits_val + bias_vec.reshape(1, -1), axis=1)
        rep = classification_report(y_val, y_pred, target_names=class_names, output_dict=True, zero_division=0)
        return rep["macro avg"]["f1-score"]

    best = macro_f1_with(b)
    for _ in range(passes):
        improved = False
        for c in range(C):
            best_b_c, best_score_c = b[c], best
            for val in np.arange(lo, hi + 1e-9, step):
                b_try = b.copy()
                b_try[c] = val
                score = macro_f1_with(b_try)
                if score > best_score_c + 1e-6:
                    best_score_c, best_b_c = score, val
            if best_b_c != b[c]:
                b[c] = best_b_c
                best = best_score_c
                improved = True
        if not improved:
            break
    return b, float(best)

CALIB_DIR2 = os.path.join(OUT_DIR, "calibration_vector")
os.makedirs(CALIB_DIR2, exist_ok=True)

for key in MODELS_TO_RUN:
    print(f"\n=== Multiclass calibration for {key} ===")

    pol_val_logits = _get_pol_logits(key, X_val[TITLE_COL].values,  X_val[TEXT_COL].values)
    pol_tst_logits = _get_pol_logits(key, X_test[TITLE_COL].values, X_test[TEXT_COL].values)

    y_val = ypol_val
    y_tst = ypol_test
    class_names = list(pol_le.classes_)

    b_vec, val_macro = coord_search_biases(pol_val_logits, y_val, class_names, passes=3, grid=(-0.8, 0.8, 0.1))
    print("Chosen bias vector (VAL macro-F1=", f"{val_macro:.3f}", "):", dict(zip(class_names, map(lambda x: round(float(x),2), b_vec))))

    # Test before/after
    y_before = np.argmax(pol_tst_logits, axis=1)
    rep_before = classification_report(y_tst, y_before, target_names=class_names, output_dict=True, zero_division=0)

    y_after = np.argmax(pol_tst_logits + b_vec.reshape(1, -1), axis=1)
    rep_after  = classification_report(y_tst, y_after, target_names=class_names, output_dict=True, zero_division=0)

    print(f"TEST macro-F1: {rep_before['macro avg']['f1-score']:.3f} → {rep_after['macro avg']['f1-score']:.3f}")
    for cname in class_names:
        b = rep_before[cname]; a = rep_after[cname]
        print(f"  {cname:>13}: P={b['precision']:.3f} R={b['recall']:.3f} F1={b['f1-score']:.3f} (n={int(b['support'])})"
              f"  →  P={a['precision']:.3f} R={a['recall']:.3f} F1={a['f1-score']:.3f} (n={int(a['support'])})")

    with open(os.path.join(CALIB_DIR2, f"{key}_bias_vector.json"), "w") as f:
        json.dump({
            "bias_vector": {class_names[i]: float(b_vec[i]) for i in range(len(class_names))},
            "val_macro_f1": val_macro,
            "test_macro_f1_before": float(rep_before["macro avg"]["f1-score"]),
            "test_macro_f1_after":  float(rep_after["macro avg"]["f1-score"])
        }, f, indent=2)



=== Multiclass calibration for mbert ===


Chosen bias vector (VAL macro-F1= 0.610 ): {'non_polarized': -0.8, 'objective': -0.8, 'partisan': -0.2}
TEST macro-F1: 0.595 → 0.617
  non_polarized: P=0.538 R=0.713 F1=0.613 (n=327)  →  P=0.625 R=0.596 F1=0.610 (n=327)
      objective: P=0.298 R=0.494 F1=0.371 (n=79)  →  P=0.331 R=0.494 F1=0.396 (n=79)
       partisan: P=0.894 R=0.722 F1=0.799 (n=820)  →  P=0.857 R=0.832 F1=0.844 (n=820)

=== Multiclass calibration for xlm_roberta ===


Chosen bias vector (VAL macro-F1= 0.630 ): {'non_polarized': -0.3, 'objective': -0.5, 'partisan': 0.0}
TEST macro-F1: 0.604 → 0.621
  non_polarized: P=0.537 R=0.703 F1=0.609 (n=327)  →  P=0.590 R=0.642 F1=0.615 (n=327)
      objective: P=0.321 R=0.544 F1=0.404 (n=79)  →  P=0.353 R=0.519 F1=0.421 (n=79)
       partisan: P=0.892 R=0.722 F1=0.798 (n=820)  →  P=0.863 R=0.794 F1=0.827 (n=820)


## SECTION 12

In [17]:
# ===== Section 12 — Length Diagnostics (clean) =====
import warnings

def token_lengths_summary(texts, titles, tokenizer, n=5000):
    # Random sample (or full if dataset is small)
    n = min(n, len(texts))
    idx = np.random.choice(len(texts), size=n, replace=False) if len(texts) > n else np.arange(len(texts))

    lengths = []
    # Silence the "sequence > 512" warnings emitted by some tokenizers for inspection
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Token indices sequence length is longer.*")
        for i in idx:
            s = f"{titles[i]} [SEP] {texts[i]}"
            # We want raw length pre-truncation to choose MAX_LENGTH wisely
            ids = tokenizer.encode(s, add_special_tokens=True, truncation=False)
            lengths.append(len(ids))

    arr = np.array(lengths)
    stats = {
        "mean": float(arr.mean()),
        "p50":  float(np.percentile(arr, 50)),
        "p90":  float(np.percentile(arr, 90)),
        "p95":  float(np.percentile(arr, 95)),
        "p99":  float(np.percentile(arr, 99)),
        "max":  int(arr.max())
    }
    print("Token length stats:", stats)
    return stats

for key in MODELS_TO_RUN:
    name = MODEL_CONFIGS[key]["name"]
    tok = AutoTokenizer.from_pretrained(name)
    print(f"\n[{key}] {name}")
    token_lengths_summary(
        texts=X_train[TEXT_COL].values,
        titles=X_train[TITLE_COL].values,
        tokenizer=tok,
        n=5000
    )

# Tip:
# If p95 is comfortably < 192, you're fine. If you see p95 > 192, consider MAX_LENGTH=224
# (Update in Section 3 if you decide to bump it.)


Token indices sequence length is longer than the specified maximum sequence length for this model (916 > 512). Running this sequence through the model will result in indexing errors



[mbert] bert-base-multilingual-cased
Token length stats: {'mean': 111.0656, 'p50': 98.0, 'p90': 183.0, 'p95': 197.0, 'p99': 234.0, 'max': 916}

[xlm_roberta] xlm-roberta-base


Token indices sequence length is longer than the specified maximum sequence length for this model (950 > 512). Running this sequence through the model will result in indexing errors


Token length stats: {'mean': 108.7404, 'p50': 97.0, 'p90': 171.0, 'p95': 184.0, 'p99': 220.0, 'max': 950}
